In [1]:
import pandas as pd
apart = pd.read_csv("서울시 공동주택 아파트 정보.csv", encoding= 'cp949')
nearStation = pd.read_csv('nearStation.csv')
nearBus = pd.read_csv('nearBus.csv')
nearSchool = pd.read_csv('mini_school.csv')

In [2]:
nearSchool = nearSchool.rename(columns={'8':'주변 학교 수','1':'번호'})
nearStation = nearStation.rename(columns ={'2':'주변 지하철역 수','1':'번호'} )
nearBus = nearBus.rename(columns = {'88':'주변 버스정류장 수','1':'번호'})

### 데이터 합치기

In [3]:
merged_df = apart.merge(nearSchool, how='left',on= '번호')
merged_df = merged_df.merge(nearStation, how='left',on= '번호')
merged_df = merged_df.merge(nearBus, how='left',on= '번호')

In [4]:
merged_df['주변 학교 수'] = merged_df['주변 학교 수'].fillna(0)
merged_df['주변 지하철역 수'] = merged_df['주변 지하철역 수'].fillna(0)
merged_df['주변 버스정류장 수'] = merged_df['주변 버스정류장 수'].fillna(0)

In [5]:
merged_df.to_csv('서울아파트.csv',index=False)

### 필요한 데이터만 추출하기

In [6]:
merged_df.columns

Index(['번호', 'k-아파트코드', 'k-아파트명', 'k-단지분류(아파트,주상복합등등)', 'kapt도로명주소',
       '주소(시도)k-apt주소split', '주소(시군구)', '주소(읍면동)', '나머지주소', '주소(도로명)',
       '주소(도로상세주소)', 'k-전화번호', 'k-팩스번호', '단지소개기존clob', '단지첨부파일',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적',
       'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', '주변 학교 수',
       '주변 지하철역 수', '주변 버스정류장 수'],
      dtype='object')

In [7]:
merged_df['k-전체세대수'].isnull().sum()

0

In [8]:
merged_df['주차대수'].isnull().sum()

39

In [9]:
merged_df['k-전체세대수'] = merged_df['k-전체세대수'].astype('int')
merged_df['주차대수'] = merged_df['주차대수'].fillna(0).astype('int')

In [10]:
merged_df['세대당 주차대수'] = [merged_df['주차대수'][i] / merged_df['k-전체세대수'][i] for i in range(len(merged_df))]

In [11]:
merged_df['연식'] = 2023 - merged_df['k-사용검사일-사용승인일'].astype('datetime64').dt.year

In [12]:
old_df = merged_df[merged_df['연식']>=30 ]

In [13]:
old_df = old_df[old_df['세대당 주차대수'] != 0]

In [14]:
old_df.to_csv('낡은아파트.csv')

In [15]:
old_df

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),...,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,주변 학교 수,주변 지하철역 수,주변 버스정류장 수,세대당 주차대수,연식
11,13,A13790701,잠원한신그린,아파트,서울특별시 서초구 신반포로33길 71,서울,서초구,잠원동,잠원한신그린아파트,신반포로33길,...,Y,N,127.0078749,37.513371,2013-03-07 09:47:01.0,6.0,4.0,49.0,0.906667,30.0
17,19,A13407201,명일삼익그린11차,아파트,서울특별시 강동구 명일로 376,서울,강동구,명일동,38번지,명일로,...,Y,N,127.1504982,37.551807,2013-03-07 09:47:02.0,16.0,3.0,88.0,1.184211,37.0
20,22,A15001001,여의도초원,아파트,서울특별시 영등포구 국회대로76가길 11,서울,영등포구,여의도동,서울시 영등포구 여의도동11-1,국회대로76가길,...,Y,Y,126.9231927,37.530964,2013-03-07 09:47:02.0,0.0,2.0,43.0,0.601307,52.0
39,41,A13486401,천호현대,아파트,서울특별시 강동구 진황도로 47-2,서울,강동구,천호동,천호현대아파트,진황도로,...,Y,N,127.1329005,37.538399,2013-03-07 09:47:04.0,5.0,4.0,96.0,0.641026,33.0
40,42,A13187501,중화극동,아파트,서울특별시 중랑구 봉화산로27길 32,서울,중랑구,중화동,19번지,봉화산로27길,...,Y,N,127.0837949,37.603652,2013-03-07 09:46:52.0,13.0,5.0,66.0,1.025641,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,40064,A10024829,대림 우성2차,아파트,서울특별시 영등포구 도림로47길 1,서울,영등포구,대림동,NaN,도림로47길,...,Y,N,126.89981,37.495928,2020-07-08 00:00:00.0,17.0,2.0,140.0,0.758333,31.0
2566,40076,A10024799,대치1차현대아파트,아파트,서울특별시 강남구 영동대로57길 28,서울,강남구,대치동,NaN,영동대로57길,...,Y,N,127.0651317,37.499746,2020-07-28 16:00:22.0,10.0,2.0,52.0,0.775000,33.0
2567,40077,A10024798,상일우성타운,연립주택,서울특별시 강동구 상암로 331,서울,강동구,상일동,NaN,상암로,...,Y,N,127.159188,37.546536,2020-07-13 00:00:00.0,7.0,0.0,48.0,1.523810,34.0
2571,40083,A10024780,금강주택,연립주택,서울특별시 종로구 평창12길 8-22,서울,종로구,평창동,NaN,평창12길,...,Y,Y,126.9689263,37.608204,2020-07-20 00:00:00.0,2.0,0.0,44.0,0.714286,31.0


In [87]:
old_df.columns

Index(['번호', 'k-아파트코드', 'k-아파트명', 'k-단지분류(아파트,주상복합등등)', 'kapt도로명주소',
       '주소(시도)k-apt주소split', '주소(시군구)', '주소(읍면동)', '나머지주소', '주소(도로명)',
       '주소(도로상세주소)', 'k-전화번호', 'k-팩스번호', '단지소개기존clob', '단지첨부파일',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적',
       'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', '주변 학교 수',
       '주변 지하철역 수', '주변 버스정류장 수', '세대당 주차대수', '연식', '가중치', '연식 정규화',
       '학교 수 정규화', '지하철역 수 정규화', '버스정류장 수 정규화', '주차대수 정규화', '재개발 점수'],
      dtype='object')

### 가중치 정의 및 적용

In [17]:
old_df['연식 정규화'] = (old_df['연식'] - old_df['연식'].min()) / (old_df['연식'].max() - old_df['연식'].min())

In [18]:
old_df['학교 수 정규화'] =(old_df['주변 학교 수'] - old_df['주변 학교 수'].min()) / (old_df['주변 학교 수'].max() - old_df['주변 학교 수'].min())

In [19]:
old_df['지하철역 수 정규화'] =(old_df['주변 지하철역 수'] - old_df['주변 지하철역 수'].min()) / (old_df['주변 지하철역 수'].max() - old_df['주변 지하철역 수'].min())

In [20]:
old_df['버스정류장 수 정규화'] = (old_df['주변 버스정류장 수'] - old_df['주변 버스정류장 수'].min()) / (old_df['주변 버스정류장 수'].max() - old_df['주변 버스정류장 수'].min())

In [21]:
old_df['주차대수 정규화'] =(old_df['세대당 주차대수'] - old_df['세대당 주차대수'].min()) / (old_df['세대당 주차대수'].max() - old_df['세대당 주차대수'].min())

In [47]:
old_df['재개발 점수' ] = 5*old_df['연식 정규화'] + old_df['학교 수 정규화']+ 3*old_df['지하철역 수 정규화']+2*old_df['버스정류장 수 정규화']-4*old_df['주차대수 정규화']

In [55]:
old_df.sort_values(by = '재개발 점수',ascending=False).to_csv('재개발_아파트.csv',index=False)

In [48]:
rank3=old_df.sort_values(by = '재개발 점수',ascending=False).iloc[:3]

In [50]:
rank3.to_csv('상위3.csv')

In [57]:
rank3

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),...,주변 버스정류장 수,세대당 주차대수,연식,가중치,연식 정규화,학교 수 정규화,지하철역 수 정규화,버스정류장 수 정규화,주차대수 정규화,재개발 점수
751,767,A13607302,안암대광,아파트,서울특별시 성북구 안암로9나길 14,서울,성북구,안암동3가,대광아파트,안암로9나길,...,94.0,0.289017,52.0,780.450867,1.000000,0.636364,0.8,0.580247,0.144339,8.619502
1080,1114,A15085805,신길남서울,아파트,서울특별시 영등포구 신길로 108,서울,영등포구,신길동,관리사무소,신길로,...,155.0,0.521236,49.0,870.061776,0.863636,0.545455,0.4,0.956790,0.260471,6.935332
802,820,A15089513,여의도진주,아파트,서울특별시 영등포구 국제금융로 108-6,서울,영등포구,여의도동,54번지,국제금융로,...,69.0,0.505319,46.0,659.265957,0.727273,0.272727,0.9,0.425926,0.252511,6.450898


In [82]:
rank3.iloc[2]['주변 버스정류장 수']

69.0

In [60]:
old_df['주변 버스정류장 수'].mean()

78.22489959839358

In [83]:
rank3.iloc[2]['주변 학교 수']

6.0

In [61]:
old_df['주변 학교 수'].mean()

10.008032128514056

In [84]:
rank3.iloc[2]['주변 지하철역 수']

9.0

In [63]:
old_df['주변 지하철역 수'].mean()

2.708835341365462

In [85]:
rank3.iloc[2]['세대당 주차대수'].mean()

0.5053191489361702

In [76]:
old_df['세대당 주차대수'].mean()

0.7474474103748691

In [86]:
rank3['연식'].iloc[2].mean()

46.0

In [74]:
old_df['연식'].mean()

35.53012048192771